# Telemetry-to-Insight: Full Pipeline Demo

End-to-end: load telemetry with cuDF, retrieve filtered data, send to NIM for natural-language answers.

**Colab:** You need NIM running on GKE with a public URL. Paste your NIM URL in the Setup cell below. See "Deploy NIM on GKE" at the end of this notebook.

<a href="https://colab.research.google.com/github/KarthikSriramGit/Project-Insight/blob/main/notebooks/03_query_telemetry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Configure runtime first:** Runtime > Change runtime type > Hardware accelerator: **GPU** or **None** > Save.

In [1]:
# Colab setup: clone repo and install dependencies (run this cell first)
try:
    import google.colab
    get_ipython().system("git clone -q https://github.com/KarthikSriramGit/Project-Insight.git")
    get_ipython().run_line_magic("cd", "Project-Insight")
    get_ipython().system("pip install -q -r requirements.txt")
except Exception:
    pass

/content/Project-Insight


## Setup

**Store your NIM IP securely using Colab Secrets (recommended):**
1. In Colab, click the **key icon** (Secrets) in the left sidebar.
2. Add a new secret: Name = `NIM_BASE_URL`, Value = `http://YOUR_EXTERNAL_IP:8000`
3. Toggle **Notebook access** to ON.

The cell below reads from Colab Secrets automatically. If no secret is set, it falls back to the `NIM_BASE_URL` environment variable or a placeholder.

In [2]:
import os
import sys
import subprocess
from pathlib import Path

ROOT = Path(".").resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

# Read NIM URL from Colab Secrets (preferred) or environment variable
try:
    from google.colab import userdata
    NIM_BASE_URL = userdata.get("NIM_BASE_URL")
except (ImportError, userdata.SecretNotFoundError):
    NIM_BASE_URL = os.environ.get("NIM_BASE_URL", "http://YOUR_EXTERNAL_IP:8000")

if "YOUR_EXTERNAL_IP" in NIM_BASE_URL:
    print("WARNING: Set NIM_BASE_URL in Colab Secrets (key icon in sidebar) or as an environment variable.")
    print("         Value should be: http://<YOUR_LOADBALANCER_IP>:8000")

data_path = ROOT / "data" / "synthetic" / "fleet_telemetry.parquet"
if not data_path.exists():
    subprocess.run([
        "python", "data/synthetic/generate_telemetry.py",
        "--rows", "100000", "--output-dir", "data/synthetic", "--format", "parquet",
    ], check=True, cwd=str(ROOT))

from src.query.engine import TelemetryQueryEngine

engine = TelemetryQueryEngine(
    data_path=str(data_path),
    nim_base_url=NIM_BASE_URL,
    max_context_rows=500,
)
print(f"Engine ready. NIM URL: {NIM_BASE_URL.split('/')[0]}//<HIDDEN>:{NIM_BASE_URL.split(':')[-1]}")

Engine ready. NIM URL: http://136.119.171.193:8000


## Query 1: Max brake pressure

In [3]:
print("=" * 60)
print("Query 1: Max brake pressure percentage across all vehicles")
print("=" * 60)
answer1 = engine.query(
    "What is the maximum brake_pressure_pct value across all vehicles? Which vehicle had it?",
    sensor_type="can",
)
print(answer1)

Query 1: Max brake pressure percentage across all vehicles
The maximum brake_pressure_pct value across all vehicles is 5.0. The vehicle that had it was V000.


## Query 2: Vehicle speed analysis

In [4]:
print("=" * 60)
print("Query 2: Average speed per vehicle")
print("=" * 60)
answer2 = engine.query(
    "What is the average vehicle_speed_kmh for each vehicle? List them from fastest to slowest.",
    sensor_type="can",
)
print(answer2)

Query 2: Average speed per vehicle
Based on the provided telemetry data, I calculated the average vehicle speed for each vehicle and listed them from fastest to slowest:

1. V000: 44.463773 km/h
2. V003: 39.803544 km/h
3. V004: 39.511652 km/h
4. V008: 37.827314 km/h
5. V005: 35.991443 km/h
6. V007: 23.575781 km/h
7. V001: 21.841788 km/h
8. V009: 18.608306 km/h
9. Other vehicles (V002, V006, V007, etc.): No data available

Note that the average speeds are calculated by taking the mean of the vehicle_speed_kmh values for each vehicle. However, it seems that some vehicles (e.g., V002, V006, etc.) do not have any data points in the provided telemetry data, so their average speeds cannot be calculated.


## Query 3: Hard braking events

In [5]:
print("=" * 60)
print("Query 3: Hard braking events (brake_pressure_pct > 90)")
print("=" * 60)
answer3 = engine.query(
    "How many rows have brake_pressure_pct above 90? What is the average vehicle_speed_kmh during these hard braking events?",
    sensor_type="can",
)
print(answer3)

Query 3: Hard braking events (brake_pressure_pct > 90)
Based on the data, there are 2 rows with brake_pressure_pct above 90.

Here are the rows:
```
41      17280864043       V009         can      NaN      NaN      NaN     NaN     NaN     NaN            NaN            NaN            NaN            NaN          NaN        NaN        NaN             NaN     None          41.546319           95.807041          105.310109              32.565705  4498.488696            4.0       NaN        NaN         NaN             NaN            NaN          NaN           NaN           NaN           NaN
54      47522376118       V008         can      NaN      NaN      NaN     NaN     NaN     NaN            NaN            NaN            NaN            NaN          NaN        NaN        NaN             NaN     None         108.423888           98.743965           93.392472              65.395676  2340.684601            5.0       NaN        NaN         NaN             NaN            NaN          NaN        

## Query 4: IMU anomaly detection

In [6]:
print("=" * 60)
print("Query 4: IMU anomaly detection")
print("=" * 60)
answer4 = engine.query(
    "Are there any unusual acceleration values in the data? Look at accel_x, accel_y, accel_z "
    "and identify any readings that seem abnormally high or low compared to the typical range.",
    sensor_type="imu",
)
print(answer4)

Query 4: IMU anomaly detection
After analyzing the data, I found a few acceleration values that seem abnormally high or low compared to the typical range.

* The highest acceleration value is -0.417785 in the z-axis (accel_z) at timestamp_ns 82084104205, which is relatively low and may indicate some unusual deceleration or braking event.
* The second highest acceleration value is 0.643591 in the x-axis (accel_x) at timestamp_ns 34561728086, which is relatively high and may indicate some unusual jerk or torque.
* The lowest acceleration value is -0.190947 in the x-axis (accel_x) at timestamp_ns 77763888194, which is also relatively low and may indicate some unusual deceleration or braking event.

Please note that these values are relatively extreme and may be worth further investigation to understand the context and cause of these unusual acceleration values.


## Query 5: LiDAR point cloud overview

In [7]:
print("=" * 60)
print("Query 5: LiDAR point cloud overview")
print("=" * 60)
answer5 = engine.query(
    "What is the average and max point_count across all LiDAR readings? "
    "What is the typical max_range observed?",
    sensor_type="lidar",
)
print(answer5)

Query 5: LiDAR point cloud overview
Based on the provided data, here are the answers to the question:

* The average point_count across all LiDAR readings is: `sum(point_count) / count(*)` = 244117.47
* The maximum point_count across all LiDAR readings is: `max(point_count)` = 444562.0
* The typical max_range observed is: `mean(max_range)` = 194.112692 (note: this is just an average, as max_range values are quite variable)


## Query 6: GPS coverage

In [8]:
print("=" * 60)
print("Query 6: GPS coverage area")
print("=" * 60)
answer6 = engine.query(
    "What is the geographic bounding box of the fleet? "
    "Give the min/max latitude and longitude values.",
    sensor_type="gps",
)
print(answer6)

Query 6: GPS coverage area
Based on the provided telemetry data, the geographic bounding box of the fleet can be defined as:

Min Latitude: 37.039091
Max Latitude: 37.760847
Min Longitude: -122.485406
Max Longitude: -121.553374

These values represent the minimum and maximum latitude and longitude values observed across all the GPS readings in the data.


## Query 7: Fleet-wide summary

In [9]:
print("=" * 60)
print("Query 7: Fleet-wide CAN bus summary")
print("=" * 60)
answer7 = engine.query(
    "Provide a brief fleet health summary: average speed, average throttle, "
    "average brake pressure percentage, and the most common gear position.",
    sensor_type="can",
)
print(answer7)

Query 7: Fleet-wide CAN bus summary
Based on the provided telemetry data, here is a brief fleet health summary:

* Average speed: The average vehicle speed is approximately 53.4 km/h, as calculated from the 500+ rows of data.
* Average throttle: Unfortunately, the throttle position is only reported in percentage (%) format, so it's challenging to provide an average value. However, we can note that the throttle position ranges from 0% to 110%, suggesting that drivers are moderately to aggressively accelerating.
* Average brake pressure percentage: Similar to the throttle position, the brake pressure is only reported in percentage (%) format, making it difficult to calculate an average value. However, we can observe that brake pressure ranges from 0% to 110%, indicating that drivers are moderately applying the brakes.
* Most common gear position: Unfortunately, the gear position data is missing in the provided dataset, so we cannot determine the most common gear position.

Please note th

## All answers summary

Collected output from all queries above for easy review.

In [10]:
queries = {
    "1. Max brake pressure (CAN)": answer1,
    "2. Avg speed per vehicle (CAN)": answer2,
    "3. Hard braking events (CAN)": answer3,
    "4. IMU anomaly detection": answer4,
    "5. LiDAR point cloud overview": answer5,
    "6. GPS coverage area": answer6,
    "7. Fleet-wide CAN summary": answer7,
}

print("=" * 60)
print("TELEMETRY INSIGHT SUMMARY")
print("=" * 60)
for title, ans in queries.items():
    print(f"\n{'─' * 60}")
    print(f"  {title}")
    print(f"{'─' * 60}")
    print(ans)

print(f"\n{'=' * 60}")
print(f"Pipeline: cuDF + UVM → NIM (Llama 3 8B) on GKE")
print(f"{'=' * 60}")

TELEMETRY INSIGHT SUMMARY

────────────────────────────────────────────────────────────
  1. Max brake pressure (CAN)
────────────────────────────────────────────────────────────
The maximum brake_pressure_pct value across all vehicles is 5.0. The vehicle that had it was V000.

────────────────────────────────────────────────────────────
  2. Avg speed per vehicle (CAN)
────────────────────────────────────────────────────────────
Based on the provided telemetry data, I calculated the average vehicle speed for each vehicle and listed them from fastest to slowest:

1. V000: 44.463773 km/h
2. V003: 39.803544 km/h
3. V004: 39.511652 km/h
4. V008: 37.827314 km/h
5. V005: 35.991443 km/h
6. V007: 23.575781 km/h
7. V001: 21.841788 km/h
8. V009: 18.608306 km/h
9. Other vehicles (V002, V006, V007, etc.): No data available

Note that the average speeds are calculated by taking the mean of the vehicle_speed_kmh values for each vehicle. However, it seems that some vehicles (e.g., V002, V006, etc.) 